# 效能指標-預測最低價格

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 讀取原始數據
file_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'
data = pd.read_csv(file_path)

# 分離經濟艙與商務艙的數據
economy_class_data = data[data["艙等"] == "經濟艙"].copy()
business_class_data = data[data["艙等"] == "商務艙"].copy()

# One-hot encoding
categorical_cols = ['星期', '出發時段', '出發機場代號', '抵達時段', '抵達機場代號',
                    '航空公司', '航空聯盟', '機型分類', '是否過夜', '是否為平日', '機型', '假期', 'Region']
economy_class_data = pd.get_dummies(economy_class_data, columns=categorical_cols, drop_first=True)
business_class_data = pd.get_dummies(business_class_data, columns=categorical_cols, drop_first=True)

# 取兩個數據集共同欄位
common_cols = list(set(economy_class_data.columns) & set(business_class_data.columns))
economy_class_data = economy_class_data[common_cols]
business_class_data = business_class_data[common_cols]

# 標準化數值變數
num_cols = ['飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
scaler = StandardScaler()
economy_class_data[num_cols] = scaler.fit_transform(economy_class_data[num_cols])
business_class_data[num_cols] = scaler.transform(business_class_data[num_cols])

# 選取建模特徵
target_keywords = ['出發時段_', '出發機場代號_', '抵達時段_', '航空聯盟_', '機型分類_', '假期_', '是否為平日_']
economy_dummy_cols = [col for col in economy_class_data.columns if any(keyword in col for keyword in target_keywords)]
business_dummy_cols = [col for col in business_class_data.columns if any(keyword in col for keyword in target_keywords)]
economy_feature_cols = economy_dummy_cols + num_cols
business_feature_cols = business_dummy_cols + num_cols

# 設定隨機種子
seed = 123

# 訓練與預測函數
def train_and_predict_model(model, X_train, X_test, y_train, y_test, model_label):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"📌 {model_label} 測試集 MSE: {mse:.4f}")
    print(f"📌 {model_label} 測試集 R²: {r2:.4f}")
    return y_pred

# 定義模型
rf_model = RandomForestRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
svr_model = SVR()
xgb_model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)

# === 經濟艙模型訓練 ===
print("====== 經濟艙模型訓練 ======")
X_economy = economy_class_data[economy_feature_cols]
y_economy = economy_class_data["最低價格_log"]
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_economy, y_economy, test_size=0.3, random_state=seed)

economy_pred_rf = train_and_predict_model(rf_model, X_train_e, X_test_e, y_train_e, y_test_e, "RF")
economy_pred_svr = train_and_predict_model(svr_model, X_train_e, X_test_e, y_train_e, y_test_e, "SVR")
economy_pred_xgb = train_and_predict_model(xgb_model, X_train_e, X_test_e, y_train_e, y_test_e, "XGB")

economy_results = economy_class_data.loc[X_test_e.index].copy()
economy_results["預測最低價格_rf"] = np.exp(economy_pred_rf)
economy_results["預測最低價格_svr"] = np.exp(economy_pred_svr)
economy_results["預測最低價格_xgb"] = np.exp(economy_pred_xgb)

# === 商務艙模型訓練 ===
print("\n====== 商務艙模型訓練 ======")
X_business = business_class_data[business_feature_cols]
y_business = business_class_data["最低價格_log"]
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_business, y_business, test_size=0.3, random_state=seed)

business_pred_rf = train_and_predict_model(rf_model, X_train_b, X_test_b, y_train_b, y_test_b, "RF")
business_pred_svr = train_and_predict_model(svr_model, X_train_b, X_test_b, y_train_b, y_test_b, "SVR")
business_pred_xgb = train_and_predict_model(xgb_model, X_train_b, X_test_b, y_train_b, y_test_b, "XGB")

business_results = business_class_data.loc[X_test_b.index].copy()
business_results["預測最低價格_rf"] = np.exp(business_pred_rf)
business_results["預測最低價格_svr"] = np.exp(business_pred_svr)
business_results["預測最低價格_xgb"] = np.exp(business_pred_xgb)

# === 計算效能 ===
def calculate_efficiency(df, pred_col, base_col="隨機購買平均價格"):
    return (df[base_col] - df[pred_col]) / df[base_col]

economy_results["效能_rf"] = calculate_efficiency(economy_results, "預測最低價格_rf")
economy_results["效能_svr"] = calculate_efficiency(economy_results, "預測最低價格_svr")
economy_results["效能_xgb"] = calculate_efficiency(economy_results, "預測最低價格_xgb")

business_results["效能_rf"] = calculate_efficiency(business_results, "預測最低價格_rf")
business_results["效能_svr"] = calculate_efficiency(business_results, "預測最低價格_svr")
business_results["效能_xgb"] = calculate_efficiency(business_results, "預測最低價格_xgb")

# 合併預測結果與效能
predicted_data = pd.concat([economy_results, business_results]).sort_index()
data_updated = data.merge(
    predicted_data[["預測最低價格_rf", "預測最低價格_svr", "預測最低價格_xgb",
                    "效能_rf", "效能_svr", "效能_xgb"]],
    left_index=True, right_index=True, how='left'
)

# 儲存最終結果
output_path = '/Users/yuchingchen/Documents/專題/efficacy/efficacy_data/short_flight_predict.csv'
data_updated.to_csv(output_path, index=False)

# 顯示平均節省比例
print("\n====== 平均節省比例 ======")
print(" 經濟艙:")
print(f"    RF  : {economy_results['效能_rf'].mean() * 100:.2f}%")
print(f"    SVR : {economy_results['效能_svr'].mean() * 100:.2f}%")
print(f"    XGB : {economy_results['效能_xgb'].mean() * 100:.2f}%")

print("\n 商務艙:")
print(f"    RF  : {business_results['效能_rf'].mean() * 100:.2f}%")
print(f"    SVR : {business_results['效能_svr'].mean() * 100:.2f}%")
print(f"    XGB : {business_results['效能_xgb'].mean() * 100:.2f}%")

print("\n✅ 預測與效能計算完成，已儲存至最終結果檔案！")

====== 經濟艙模型訓練 ======
📌 RF 測試集 MSE: 0.0472
📌 RF 測試集 R²: 0.8401
📌 SVR 測試集 MSE: 0.0515
📌 SVR 測試集 R²: 0.8257
📌 XGB 測試集 MSE: 0.0446
📌 XGB 測試集 R²: 0.8488

====== 商務艙模型訓練 ======
📌 RF 測試集 MSE: 0.0226
📌 RF 測試集 R²: 0.7561
📌 SVR 測試集 MSE: 0.0235
📌 SVR 測試集 R²: 0.7469
📌 XGB 測試集 MSE: 0.0240
📌 XGB 測試集 R²: 0.7415

====== 平均節省比例 ======
 經濟艙:
    RF  : 20.13%
    SVR : 20.37%
    XGB : 19.88%

 商務艙:
    RF  : 8.54%
    SVR : 8.35%
    XGB : 8.71%

✅ 預測與效能計算完成，已儲存至最終結果檔案！
